Imports and functions

In [98]:
import os
import netCDF4 as nc4
import numpy as np
#import numpy.ma as ma
%matplotlib
import matplotlib as mpl
import matplotlib.pyplot as plt

Using matplotlib backend: TkAgg


User inputs

In [99]:
indir = '/home/storm/brodzik/Data/CSH/2018/10'
VERT_RES = 0.125  #km
STRA = 1
CONV = 2
THRESHOLD = 0.0  #dBZ
DELTA_HT_FOR_SLOPE = 1.5 #km
SLOPE = 7.5
MAX_BB_HT = 5.0          #km
REFL_MAX_OFFSET = 2.0    #dBZ

In [100]:
fname = 'GPM2Ku5_uw2_20181001.004259_to_20181001.004807_026079_CSH.nc'
#fname = 'GPM2Ku5_uw2_20181001.021447_to_20181001.022042_026080_CSH.nc'
#fname = 'GPM2Ku5_uw2_20181001.055925_to_20181001.060340_026082_CSH.nc'
#fname = 'GPM2Ku5_uw2_20181001.192142_to_20181001.192756_026091_CSH.nc'

In [101]:
ncid = nc4.Dataset(indir+'/'+fname,'r')
    
refl = np.array(ncid.variables['refl'])
(ntime,nalt,nlat,nlon) = refl.shape
refl_missing = ncid.variables['refl']._FillValue
refl[:,0,:,:] = refl_missing  # get rid of lowest level due to suspicious data

bin_missing = int(refl_missing)
    
rt = np.array(ncid.variables['rain_type'])
rt_missing = ncid.variables['rain_type']._FillValue

# use if looking for uw_rt
#rt_uw = np.array(rt, copy=True)
# if rt_uw already exists
rt_uw = np.array(ncid.variables['rain_type_uw'])
rt_uw_missing = ncid.variables['rain_type_uw']._FillValue

srt = np.array(ncid.variables['shallow_rain_type'])
srt_missing = ncid.variables['shallow_rain_type']._FillValue

In [102]:
# find max_refl and max_refl_ht at each lat/lon
max_refl = np.zeros(rt.shape,dtype=float)+refl_missing
max_refl = np.amax(refl,axis=1)
max_refl_bin = np.zeros(rt.shape,dtype=int)+bin_missing
max_refl_bin = np.argmax(refl,axis=1)
max_refl_ht = max_refl_bin * VERT_RES

max_refl_bin[(max_refl==refl_missing)] = bin_missing
max_refl_ht[(max_refl==refl_missing)] = refl_missing

# initialize reference refl/bin/ht
# these reflect highest refl,bin,ht above max_refl,bin.ht and within +/- REFL_MAX_OFFSET
reference_refl = np.array(max_refl, copy=True)
reference_refl_bin = np.array(max_refl_bin, copy=True)
reference_refl_ht = np.array(max_refl_ht, copy=True)
                                
# find bin & height of highest good refl at each lat/lon
max_ht_bin = np.zeros(rt.shape,dtype=int) + bin_missing
for lev in range(nalt-1,-1,-1):
    max_ht_bin = np.where(np.logical_and((refl[:,lev,:,:] != refl_missing),
                                         (max_ht_bin == bin_missing) ), 
                          lev, max_ht_bin ) 
max_ht = max_ht_bin * (max_ht_bin != bin_missing) * VERT_RES
max_ht[(max_ht_bin==bin_missing)] = refl_missing    
    
# For rt_uw, change rain_type to stratiform if:
# 1. rain_type = convective and
# 2. shallow_rain_type = 0 and
# 3. max_refl in column >= THRESHOLD and   ## DON'T USE THIS CRITERIA ANYMORE
# 4. [(dbz @ reference point) - (dbz @ point 1.5km higher)]/
#    [(ht  @ reference point) - (ht  @ point 1.5km higher)] >= abs(SLOPE)
#  as long as ht @ reference point <= MAX_BB_HT
                
# Determine indices of interest 
#change = np.logical_and(np.logical_and(rt==2,srt==0),max_refl>=THRESHOLD)
change = np.logical_and(rt==2,srt==0)
ind_time,ind_lat,ind_lon = np.where(change==True)
npixels = ind_time.size

In [103]:
#for ind in range(0,npixels):
#itime = ind_time[ind]
#ilat = ind_lat[ind]
#ilon = ind_lon[ind]
itime = 0
#orbit 026079
ilon = 38
ilat = 108

In [104]:
# Run test for rt_uw
print 'refl=',refl[0,:,ilat,ilon]
print 'rt=',rt[0,ilat,ilon],' rt_uw=',rt_uw[0,ilat,ilon]
print '[itime,ilat,ilon] = [',itime,',',ilat,',',ilon,']'
print 'max_refl=',max_refl[itime,ilat,ilon],' max_refl_bin=',max_refl_bin[itime,ilat,ilon],' max_refl_ht=',max_refl_ht[itime,ilat,ilon]
if max_refl[itime,ilat,ilon] != refl_missing:
                        
    next_bin = reference_refl_bin[itime,ilat,ilon]+1
    next_ht = reference_refl_ht[itime,ilat,ilon]+VERT_RES
    next_refl = refl[itime,next_bin,ilat,ilon]
    print 'next_bin=',next_bin,' next_ht=',next_ht,' next_refl=',next_refl

    while abs(max_refl[itime,ilat,ilon]-next_refl) <= REFL_MAX_OFFSET:
        reference_refl[itime,ilat,ilon]=next_refl
        reference_refl_bin[itime,ilat,ilon] = next_bin
        reference_refl_ht[itime,ilat,ilon]=next_ht  
        next_bin = reference_refl_bin[itime,ilat,ilon]+1
        next_ht = reference_refl_ht[itime,ilat,ilon]+VERT_RES
        next_refl = refl[itime,next_bin,ilat,ilon]
        print 'ref_refl=',reference_refl[itime,ilat,ilon],' ref_refl_bin=',reference_refl_bin[itime,ilat,ilon],' ref_refl_ht=',reference_refl_ht[itime,ilat,ilon]
        print 'next_bin=',next_bin,' next_ht=',next_ht,' next_refl=',next_refl
        print 'max_refl-next_refl=',abs(max_refl[itime,ilat,ilon]-next_refl)
                       
    if reference_refl_ht[itime,ilat,ilon] < MAX_BB_HT:
        if (reference_refl_ht[itime,ilat,ilon] + DELTA_HT_FOR_SLOPE) <= max_ht[itime,ilat,ilon]:
            compare_bin = reference_refl_bin[itime,ilat,ilon] + int(DELTA_HT_FOR_SLOPE/VERT_RES)
            if refl[itime,compare_bin,ilat,ilon] != refl_missing:
                compare_ht = reference_refl_ht[itime,ilat,ilon] + DELTA_HT_FOR_SLOPE
                compare_refl = refl[itime,compare_bin,ilat,ilon]
            else:
                while refl[itime,compare_bin,ilat,ilon] == refl_missing:
                    compare_bin = compare_bin-1
                    compare_ht = compare_ht-VERT_RES
                compare_refl=refl[itime,compare_bin,ilat,ilon]
                    
        else:
            compare_bin = max_ht_bin[itime,ilat,ilon]
            compare_ht = max_ht[itime,ilat,ilon]
            compare_refl = refl[itime,compare_bin,ilat,ilon]
        print ''
        print 'ref_refl_bin=',reference_refl_bin[itime,ilat,ilon],' ref_refl_ht=',reference_refl_ht[itime,ilat,ilon],' ref_refl=',reference_refl[itime,ilat,ilon]
        print 'compare_bin=',compare_bin,' compare_ht=',compare_ht,' compare_refl=',compare_refl
        if (abs(reference_refl_ht[itime,ilat,ilon] - compare_ht)) != 0.0:
            if ( abs(reference_refl[itime,ilat,ilon] - compare_refl)/
                 abs(reference_refl_ht[itime,ilat,ilon] - compare_ht) ) >= SLOPE:
                #rt_uw[itime,ilat,ilon] = STRA
                print 'Change from CONV to STRA'
            else:
                print 'Do not change from CONV to STRA'

refl= [-99.    35.08  35.08  35.08  35.08  35.07  35.07  35.06  35.07  35.07
  35.73  36.41  37.04  37.27  37.54  37.08  36.95  35.02  35.71  34.94
  32.44  29.76  27.51  25.71  25.31  23.15  20.66  18.23 -99.   -99.
 -99.   -99.   -99.   -99.   -99.   -99.   -99.   -99.   -99.   -99.
 -99.   -99.   -99.   -99.   -99.   -99.   -99.   -99.   -99.   -99.
 -99.   -99.   -99.   -99.   -99.   -99.   -99.   -99.   -99.   -99.
 -99.   -99.   -99.   -99.   -99.   -99.   -99.   -99.   -99.   -99.
  14.77  15.75  15.8   16.63  16.54  17.22  14.74 -99.   -99.   -99.
 -99.   -99.   -99.   -99.   -99.   -99.   -99.   -99.   -99.   -99.
 -99.   -99.   -99.   -99.   -99.   -99.   -99.   -99.   -99.   -99.
 -99.   -99.   -99.   -99.   -99.   -99.   -99.   -99.   -99.   -99.
 -99.   -99.   -99.   -99.   -99.   -99.   -99.   -99.   -99.   -99.
 -99.   -99.   -99.   -99.   -99.   -99.   -99.   -99.   -99.   -99.
 -99.   -99.   -99.   -99.   -99.   -99.   -99.   -99.   -99.   -99.
 -99.   -99.   -99.   -9

In [ ]:
# If rain_type_uw exists in file, overwrite it.  If not, create new variable 
try:
    ncid.variables['rain_type_uw']
    ncid.variables['rain_type_uw'][:] = rt_uw
    #print "Wrote modified var to file"
except:
    rt_uw_id = ncid.createVariable('rain_type_uw','f4',
                                  ('time','lat','lon'),zlib=True,fill_value=rt_missing)
    rt_uw_id.units = "none"
    rt_uw_id.long_name = "Rain Type UW"
    rt_uw_id.stratiform = 1
    rt_uw_id.convective = 2
    rt_uw_id.other = 3
    rt_uw_id[:,:,:] = rt_uw
    #print "Wrote new var to file"

In [105]:
ncid.close()

In [ ]:
print 'refl  = ',refl[0,:,108,38]
print 'rt    = ',rt[0,108,38]
print 'rt_uw = ',rt_uw[0,108,38]

In [111]:
# PLOT COLUMN
#some computations
delta_refl = reference_refl[itime,ilat,ilon] - compare_refl
delta_ht = compare_ht - reference_refl_ht[itime,ilat,ilon]
slope = delta_refl/delta_ht

#y = range(1, 30)
x = refl[0,0:54,ilat,ilon]
##x = refl[0,0:79,ilat,ilon]
y = np.linspace(0.0,6.625,num=54)
##y = np.linspace(0.0,6.625,num=79)
fig = plt.figure()
plt.plot(x, y, '-',color='black')
#plt.plot(x, y, 'o')
#plt.xlim(15, 50);
#plt.xlim(15, 60);
plt.xlim(10, 55);
plt.ylim(0.25, 7.0);
plt.xlabel('refl (dBZ)')
plt.ylabel('ht (km)')

#x1 = range(15,61)
x1 = range(10,56)
y1 = np.array([MAX_BB_HT for i in xrange(len(x1))])
#plt.plot(x1, y1, 'r-',label='MAX_BB_HT') 
#plt.plot()

#x2 = range(15,61)
x2 = range(10,56)
y2 = np.array([max_refl_ht[itime,ilat,ilon] for i in xrange(len(x2))])
plt.plot(x2, y2, 'm-', label='max refl pt') 
plt.plot()

#x3 = range(15,61)
x3 = range(10,56)
y3 = np.array([reference_refl_ht[itime,ilat,ilon] for i in xrange(len(x3))])
plt.plot(x3, y3, 'b--', label='ref refl pt') 
plt.plot()

#x4 = range(15,61)
x4 = range(10,56)
y4 = np.array([compare_ht for i in xrange(len(x4))])
plt.plot(x4, y4, 'g--', label='compare refl pt') 
plt.plot()

y5 = range(0,8)
x5 = np.array([reference_refl[itime,ilat,ilon] for i in xrange(len(y5))])
plt.plot(x5, y5, 'b--') 
plt.plot()

y6 = range(0,8)
x6 = np.array([compare_refl for i in xrange(len(y6))])
plt.plot(x6, y6, 'g--') 
plt.plot()

plt.legend()

plt.fill_between(x1,y1,color='gray',alpha=0.3)

plt.title('Reclassified Stratiform Reflectivity Profile - 20181001/004259')
#plt.title('Reclassified Stratiform Reflectivity Profile - 20181001/021447')
#plt.title('Reclassified Stratiform Reflectivity Profile - 20181001/055925')
#plt.title('Reclassified Stratiform Reflectivity Profile - 20181001/192142')

plt.text(18,reference_refl_ht[itime,ilat,ilon]+delta_ht/2,'Delta_ht='+str(round(delta_ht,2)))
plt.text(compare_refl+delta_refl/2,5,'Delta_refl='+str(round(delta_refl,2)),rotation=90)
plt.text(18,0.5,'SLOPE='+str(round(delta_refl/delta_ht,2)))

Text(18,0.5,'SLOPE=16.64')

In [ ]:
reference_refl[itime,ilat,ilon]

In [ ]:
compare_refl

In [93]:
(36.95-18.23)/(3.25-1.875)

13.614545454545457